# Entrenamiento de Modelo con Adam (Adaptive Moment Estimation)

Este notebook implementa un modelo de regresión usando **Adam (Adaptive Moment Estimation)** para predecir valores basados en datos de series temporales del archivo `dryer.dat`.

## ¿Qué es Adam?

Adam es un algoritmo de optimización adaptativo que combina las ideas de **Momentum** y **RMSprop**. Mantiene promedios móviles exponenciales tanto del gradiente (primer momento) como del gradiente al cuadrado (segundo momento), y además incluye corrección de sesgo.

### Características clave:
- **Combina Momentum + RMSprop**: Lo mejor de ambos mundos
- **Corrección de sesgo**: Corrige la inicialización en cero
- **Learning rates adaptativos**: Por parámetro
- **Muy popular**: El optimizador más usado en deep learning

## División de datos:
- **70%** - Entrenamiento (Train)
- **20%** - Validación (Valid)
- **10%** - Prueba (Test)

**Importante**: Los datos se dividen de forma secuencial para respetar el orden temporal.

## 1. Importar Librerías

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler

## 2. Cargar y Explorar los Datos

In [2]:
# Cargar datos desde dryer.dat
print("Cargando datos desde dryer.dat...")
data = np.loadtxt('dryer.dat')

# Separar características (X) y variable objetivo (y)
X = data[:, 0].reshape(-1, 1)  # Primera columna como entrada
y = data[:, 1].reshape(-1, 1)  # Segunda columna como salida

print(f"Datos cargados: {len(X)} muestras")
print(f"Forma de X: {X.shape}")
print(f"Forma de y: {y.shape}")

Cargando datos desde dryer.dat...
Datos cargados: 1000 muestras
Forma de X: (1000, 1)
Forma de y: (1000, 1)


## 3. División Secuencial de los Datos

Como son datos de series temporales, NO mezclamos los datos. Mantenemos el orden temporal.

In [3]:
# División de datos SECUENCIAL (para series temporales): 70% train, 20% valid, 10% test
# No mezclamos los datos, mantenemos el orden temporal
n_samples = len(X)
train_end = int(0.7 * n_samples)
valid_end = int(0.9 * n_samples)

# División secuencial
X_train = X[:train_end]
y_train = y[:train_end]

X_valid = X[train_end:valid_end]
y_valid = y[train_end:valid_end]

X_test = X[valid_end:]
y_test = y[valid_end:]

print(f"\nDivisión de datos SECUENCIAL (series temporales):")
print(f"Train: {len(X_train)} muestras ({len(X_train)/len(X)*100:.1f}%) - Índices [0:{train_end}]")
print(f"Valid: {len(X_valid)} muestras ({len(X_valid)/len(X)*100:.1f}%) - Índices [{train_end}:{valid_end}]")
print(f"Test:  {len(X_test)} muestras ({len(X_test)/len(X)*100:.1f}%) - Índices [{valid_end}:{n_samples}]")


División de datos SECUENCIAL (series temporales):
Train: 700 muestras (70.0%) - Índices [0:700]
Valid: 200 muestras (20.0%) - Índices [700:900]
Test:  100 muestras (10.0%) - Índices [900:1000]


## 4. Normalización/Estandarización de los Datos

In [4]:
# Normalizar datos usando StandardScaler
scaler_X = StandardScaler()
scaler_y = StandardScaler()

X_train_norm = scaler_X.fit_transform(X_train)
X_valid_norm = scaler_X.transform(X_valid)
X_test_norm = scaler_X.transform(X_test)

y_train_norm = scaler_y.fit_transform(y_train)
y_valid_norm = scaler_y.transform(y_valid)
y_test_norm = scaler_y.transform(y_test)

print("Datos normalizados exitosamente")

Datos normalizados exitosamente


## 5. Implementación del Modelo con Adam

Implementamos desde cero un modelo de regresión usando **Adam (Adaptive Moment Estimation)**.

### Fórmula de Adam:
```
m_t = β₁ * m_{t-1} + (1-β₁) * ∇L           # Primer momento (momentum)
v_t = β₂ * v_{t-1} + (1-β₂) * (∇L)²        # Segundo momento (RMSprop)
m̂_t = m_t / (1 - β₁ᵗ)                      # Corrección de sesgo
v̂_t = v_t / (1 - β₂ᵗ)                      # Corrección de sesgo
θ_t = θ_{t-1} - α * m̂_t / (√v̂_t + ε)
```

Donde:
- `m_t`: promedio móvil del gradiente (momentum)
- `v_t`: promedio móvil del gradiente al cuadrado (RMSprop)
- `β₁`: típicamente 0.9
- `β₂`: típicamente 0.999
- `ε`: típicamente 1e-8

In [5]:
# Implementación de Adam (Adaptive Moment Estimation)
class AdamRegressor:
    def __init__(self, learning_rate=0.001, n_epochs=700, batch_size=32, beta1=0.9, beta2=0.999, epsilon=1e-8):
        self.learning_rate = learning_rate
        self.n_epochs = n_epochs
        self.batch_size = batch_size
        self.beta1 = beta1  # Factor de decaimiento para primer momento
        self.beta2 = beta2  # Factor de decaimiento para segundo momento
        self.epsilon = epsilon
        self.weights = None
        self.bias = None
        self.m_w = None  # Primer momento para pesos (momentum)
        self.m_b = None  # Primer momento para bias
        self.v_w = None  # Segundo momento para pesos (RMSprop)
        self.v_b = None  # Segundo momento para bias
        self.t = 0  # Contador de iteraciones (para corrección de sesgo)
        self.train_losses = []
        self.valid_losses = []
        
    def _initialize_parameters(self, n_features):
        """Inicializar pesos, bias y momentos"""
        self.weights = np.random.randn(n_features, 1) * 0.01
        self.bias = np.zeros((1, 1))
        # Inicializar momentos en cero
        self.m_w = np.zeros((n_features, 1))
        self.m_b = np.zeros((1, 1))
        self.v_w = np.zeros((n_features, 1))
        self.v_b = np.zeros((1, 1))
        self.t = 0
    
    def _compute_loss(self, y_true, y_pred):
        """Calcular MSE (Mean Squared Error)"""
        return np.mean((y_true - y_pred) ** 2)
    
    def _forward(self, X):
        """Propagación hacia adelante"""
        return np.dot(X, self.weights) + self.bias
    
    def _backward(self, X, y_true, y_pred):
        """Propagación hacia atrás (calcular gradientes)"""
        n_samples = X.shape[0]
        dw = (-2/n_samples) * np.dot(X.T, (y_true - y_pred))
        db = (-2/n_samples) * np.sum(y_true - y_pred)
        return dw, db
    
    def fit(self, X_train, y_train, X_valid=None, y_valid=None):
        """Entrenar el modelo usando Adam"""
        n_samples, n_features = X_train.shape
        self._initialize_parameters(n_features)
        
        for epoch in range(self.n_epochs):
            X = X_train
            Y = y_train
            
            for i in range(0, n_samples, self.batch_size):
                X_batch = X[i:i+self.batch_size]
                y_batch = Y[i:i+self.batch_size]
                
                # Incrementar contador de tiempo
                self.t += 1
                
                # Forward pass
                y_pred = self._forward(X_batch)
                
                # Backward pass
                dw, db = self._backward(X_batch, y_batch, y_pred)
                
                # Actualizar primer momento (promedio móvil del gradiente)
                self.m_w = self.beta1 * self.m_w + (1 - self.beta1) * dw
                self.m_b = self.beta1 * self.m_b + (1 - self.beta1) * db
                
                # Actualizar segundo momento (promedio móvil del gradiente al cuadrado)
                self.v_w = self.beta2 * self.v_w + (1 - self.beta2) * (dw ** 2)
                self.v_b = self.beta2 * self.v_b + (1 - self.beta2) * (db ** 2)
                
                # Corrección de sesgo
                m_w_hat = self.m_w / (1 - self.beta1 ** self.t)
                m_b_hat = self.m_b / (1 - self.beta1 ** self.t)
                v_w_hat = self.v_w / (1 - self.beta2 ** self.t)
                v_b_hat = self.v_b / (1 - self.beta2 ** self.t)
                
                # Actualizar parámetros
                self.weights -= self.learning_rate * m_w_hat / (np.sqrt(v_w_hat) + self.epsilon)
                self.bias -= self.learning_rate * m_b_hat / (np.sqrt(v_b_hat) + self.epsilon)
            
            # Calcular pérdida en train
            y_train_pred = self._forward(X_train)
            train_loss = self._compute_loss(y_train, y_train_pred)
            self.train_losses.append(train_loss)
            
            # Calcular pérdida en validación
            if X_valid is not None and y_valid is not None:
                y_valid_pred = self._forward(X_valid)
                valid_loss = self._compute_loss(y_valid, y_valid_pred)
                self.valid_losses.append(valid_loss)
                
                if (epoch + 1) % 10 == 0:
                    print(f"Época {epoch+1}/{self.n_epochs} - Train Loss: {train_loss:.6f} - Valid Loss: {valid_loss:.6f}")
            else:
                if (epoch + 1) % 10 == 0:
                    print(f"Época {epoch+1}/{self.n_epochs} - Train Loss: {train_loss:.6f}")
    
    def predict(self, X):
        """Hacer predicciones"""
        return self._forward(X)
    
    def evaluate(self, X, y):
        """Evaluar el modelo"""
        y_pred = self.predict(X)
        mse = self._compute_loss(y, y_pred)
        rmse = np.sqrt(mse)
        
        # R² score
        ss_tot = np.sum((y - np.mean(y)) ** 2)
        ss_res = np.sum((y - y_pred) ** 2)
        r2 = 1 - (ss_res / ss_tot)
        
        return {'MSE': mse, 'RMSE': rmse, 'R2': r2}

print("Clase AdamRegressor definida exitosamente")

Clase AdamRegressor definida exitosamente


## 6. Entrenar el Modelo con Adam

### Hiperparámetros:
- **learning_rate**: 0.001
- **n_epochs**: 200
- **batch_size**: 32
- **beta1**: 0.9
- **beta2**: 0.999
- **epsilon**: 1e-8

In [6]:
# Crear y entrenar el modelo con Adam
print("="*60)
print("ENTRENAMIENTO DEL MODELO ADAM")
print("="*60)

model = AdamRegressor(learning_rate=0.001, n_epochs=200, batch_size=32, beta1=0.9, beta2=0.999, epsilon=1e-8)
model.fit(X_train_norm, y_train_norm, X_valid_norm, y_valid_norm)

ENTRENAMIENTO DEL MODELO ADAM
Época 10/200 - Train Loss: 0.977719 - Valid Loss: 0.858644
Época 20/200 - Train Loss: 0.972751 - Valid Loss: 0.859865
Época 30/200 - Train Loss: 0.971473 - Valid Loss: 0.861639
Época 40/200 - Train Loss: 0.971158 - Valid Loss: 0.862771
Época 50/200 - Train Loss: 0.971083 - Valid Loss: 0.863358
Época 60/200 - Train Loss: 0.971067 - Valid Loss: 0.863630
Época 70/200 - Train Loss: 0.971064 - Valid Loss: 0.863745
Época 80/200 - Train Loss: 0.971064 - Valid Loss: 0.863786
Época 90/200 - Train Loss: 0.971065 - Valid Loss: 0.863797
Época 100/200 - Train Loss: 0.971065 - Valid Loss: 0.863795
Época 110/200 - Train Loss: 0.971065 - Valid Loss: 0.863790
Época 120/200 - Train Loss: 0.971066 - Valid Loss: 0.863785
Época 130/200 - Train Loss: 0.971066 - Valid Loss: 0.863781
Época 130/200 - Train Loss: 0.971066 - Valid Loss: 0.863781
Época 140/200 - Train Loss: 0.971066 - Valid Loss: 0.863778
Época 140/200 - Train Loss: 0.971066 - Valid Loss: 0.863778
Época 150/200 - Tra

## 7. Evaluación del Modelo

### 7.1 Evaluación en Train

In [7]:
# Evaluar en conjunto de entrenamiento
print("\n" + "="*60)
print("EVALUACIÓN EN CONJUNTO DE ENTRENAMIENTO")
print("="*60)
train_metrics = model.evaluate(X_train_norm, y_train_norm)
for metric, value in train_metrics.items():
    print(f"{metric}: {value:.6f}")


EVALUACIÓN EN CONJUNTO DE ENTRENAMIENTO
MSE: 0.971066
RMSE: 0.985427
R2: 0.028934


### 7.2 Evaluación en Validación

In [8]:
# Evaluar en conjunto de validación
print("\n" + "="*60)
print("EVALUACIÓN EN CONJUNTO DE VALIDACIÓN")
print("="*60)
valid_metrics = model.evaluate(X_valid_norm, y_valid_norm)
for metric, value in valid_metrics.items():
    print(f"{metric}: {value:.6f}")


EVALUACIÓN EN CONJUNTO DE VALIDACIÓN
MSE: 0.863771
RMSE: 0.929393
R2: -0.066524


### 7.3 Evaluación en Test

In [9]:
# Evaluar en conjunto de prueba
print("\n" + "="*60)
print("EVALUACIÓN EN CONJUNTO DE PRUEBA (TEST)")
print("="*60)
test_metrics = model.evaluate(X_test_norm, y_test_norm)
for metric, value in test_metrics.items():
    print(f"{metric}: {value:.6f}")


EVALUACIÓN EN CONJUNTO DE PRUEBA (TEST)
MSE: 0.747634
RMSE: 0.864658
R2: -0.053959
